<div>
    <div style="float:left;">
        <img src="http://oproject.org/tiki-download_file.php?fileId=8&display&x=450&y=128" width="50%" />
    </div>
    <div style="float:left;">
        <img src="http://gfif.udea.edu.co/root/tmva/img/tmva_logo.gif" width="50%"/>
    </div>
</div>

# Graphical interface for booking DNN
<hr style="border-top-width: 4px; border-top-color: #34609b;">

<!--<script src="JsRoot/scripts/JSRootCore.js?jq2d&onload=JsRootLoadedCall" type="text/javascript"></script>-->

In [ ]:
import ROOT
from ROOT import TFile, TMVA, TCut

## Enable JS visualization

In [ ]:
%jsmva on

## Declarations, building training and testing trees

<b>For more details please see <a href="ROOTbooks-TMVA-JsMVA-UserInterface.ipynb">this</a> notebook.</b>

In [ ]:
outputFile = TFile( "TMVA.root", 'RECREATE' )

TMVA.Tools.Instance()

factory = TMVA.Factory(JobName="TMVAClassification", TargetFile=outputFile,
            V=False, Color=True, DrawProgressBar=True, Transformations=["I", "D", "P", "G","D"],
                       AnalysisType="Classification")

dataset = "tmva_class_example"
loader  = TMVA.DataLoader(dataset)

loader.AddVariable( "myvar1 := var1+var2", 'F' )
loader.AddVariable( "myvar2 := var1-var2", "Expression 2", 'F' )
loader.AddVariable( "var3",                "Variable 3", 'F' )
loader.AddVariable( "var4",                "Variable 4", 'F' )

loader.AddSpectator( "spec1:=var1*2",  "Spectator 1",  'F' )
loader.AddSpectator( "spec2:=var1*3",  "Spectator 2",  'F' )

if ROOT.gSystem.AccessPathName( "./tmva_class_example.root" ) != 0: 
    ROOT.gSystem.Exec( "wget https://root.cern.ch/files/tmva_class_example.root")
    
input = TFile.Open( "./tmva_class_example.root" )

# Get the signal and background trees for training
signal      = input.Get( "TreeS" )
background  = input.Get( "TreeB" )
    
# Global event weights (see below for setting event-wise weights)
signalWeight     = 1.0
backgroundWeight = 1.0

mycuts = TCut("")
mycutb = TCut("")

loader.AddSignalTree(signal, signalWeight)
loader.AddBackgroundTree(background, backgroundWeight)
loader.fSignalWeight = signalWeight
loader.fBackgroundWeight = backgroundWeight
loader.fTreeS = signal
loader.fTreeB = background

loader.PrepareTrainingAndTestTree(SigCut=mycuts, BkgCut=mycutb,
            nTrain_Signal=0, nTrain_Background=0, SplitMode="Random", NormMode="NumEvents", V=False)

## Booking DNN

Instead of using BookMethod for booking DNN you can use the BookDNN function. This function provides a graphical interface for building a deep neural network. 

<h4>Usage</h4>
You start with an input and output layer. By clicking on "Options" in output layer you can choose the activation function (default is "LINEAR") and you can specify the training strategy. You can add new layers by using "Add layer" menu entry. In new layers the "Options" button will provide a form to set the number of neurons (default value is 0), and another form to set the training strategy. <b>You must build the structure of network:</b> starting from input layer you have to connect all the layers. The last layer must be the output layer. To connect two layer you have to double click on first layer, grab the triangle and move inside the other layer (if you double click on triangle it will disappear). The "Scale colors" button will change the color of the layers: layer with the least neuron will be steel blue, layer with the most neuron will be red, other layers color will be scaled between this two color (by default: layer with 0 neuron is steel blue, layer with 100 neuron is red). By clicking on "Save Network" button, the network will be booked. 

The arguments of this function:
<table>
   <tr><th>Keyword</th><th>Can be used as positional argument</th><th>Default</th><th>Description</th></tr>
   <tr>
       <td>loader</td>
       <td>yes, 1.</td>
       <td>-</td>
       <td>TMVA.DataLoader object</td>
   </tr>
      <tr>
       <td>title</td>
       <td>yes, 2.</td>
       <td>"DNN"</td>
       <td>Method title</td>
   </tr>
</table>

In [ ]:
factory.BookDNN(loader)

## Train Methods

In [ ]:
factory.TrainAllMethods()

## Draw Deep Neural Network

In [ ]:
factory.DrawNeuralNetwork(dataset, "DNN")

## Close the factory's output file

In [ ]:
outputFile.Close()